In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain.schema import Document
import uuid
import re
import pandas as pd

from langchain_qdrant import QdrantVectorStore

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from dotenv import load_dotenv
load_dotenv()


True

In [4]:
files = os.listdir("../docs")

In [95]:
FILE_PATH = "docs/"+files[0]
file_id = uuid.uuid4().hex
# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

In [5]:
def clean_text(text):
    text = text.replace("&lt", "").replace("&amp", "&").replace("&quot", '"').replace("&apos", "'").replace("&nbsp", " ").replace("&gt;", ">").replace("&gt", ">").replace("&lt;", "<").replace("&lt", "<").replace("&#39;", "'").replace("&#39", "'").replace("&#34;", '"').replace("&#34", '"').replace("&gt;", ">").replace("&gt", ">").replace("&lt;", "<").replace("&lt", "<").replace("&nbsp;", " ").replace("&nbsp", " ").replace("&amp;", "&").replace("&amp", "&").replace("&quot;", '"').replace("&quot", '"').replace("&apos;", "'").replace("&apos", "'").replace("&#39;", "'").replace("&#39", "'").replace("&#34;", '"').replace("&#34", '"')
    text = re.sub(r"\s{2,}", " ", text)
    new_text = []
    for t in text.split("\n"):
        if "삭제" not in t:
            new_text.append(t)
    new_text = "\n".join(new_text)
    PAIR_PATTERN = re.compile(r"<[^<>]*>|\[[^\[\]]*\]")
    PAIR_PATTERN.sub("", new_text)
    return PAIR_PATTERN.sub("", new_text)

In [6]:
chunk_size=512
    
# 법률문서 구분자 (우선순위 순)
legal_separators = [
    "\n제\d+조",      # 조문
    "\n제\d+장",      # 장
    "\n제\d+절",      # 절
    "\n①",           # 항
    "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
    "\n1.", "\n2.", "\n3.", "\n4.", "\n5.",  # 숫자 항목
    "\n가.", "\n나.", "\n다.", "\n라.", "\n마.",  # 한글 항목
    "\n\n",          # 단락
    "\n",            # 줄바꿈
    " ",             # 공백
]

base_splitter = RecursiveCharacterTextSplitter(
    separators=legal_separators,
    chunk_size=chunk_size,
    length_function=len,
)

In [7]:
def extract_legal_structure(text):
    """텍스트에서 법률 구조 정보 추출"""
    structure = {}
    
    # 조문 찾기
    article_match = re.search(r'제(\d+)조', text)
    if article_match:
        structure['article'] = f"제{article_match.group(1)}조"
    
    # 장 찾기
    chapter_match = re.search(r'제(\d+)장', text)
    if chapter_match:
        structure['chapter'] = f"제{chapter_match.group(1)}장"
    
    # 절 찾기
    section_match = re.search(r'제(\d+)절', text)
    if section_match:
        structure['section'] = f"제{section_match.group(1)}절"
    
    # 항 찾기
    paragraph_matches = re.findall(r'[①②③④⑤⑥⑦⑧⑨⑩]', text)
    if paragraph_matches:
        structure['paragraph'] = paragraph_matches
    
    return structure

In [9]:
def _merge_short_chunks(splits, page_num, min_length=200):
    """500글자 이하의 짧은 청크들을 합치기"""
    if not splits:
        return []
    
    merged_splits = []
    current_chunk = None
    
    for split in splits:
        if current_chunk is None:
            current_chunk = split
        else:
            if len(current_chunk.page_content) < min_length:
                # 내용 합치기
                current_chunk.page_content += "\n\n" + split.page_content
                
                # 메타데이터 업데이트 (법률 구조 정보는 첫 번째 청크 기준)
                current_chunk.metadata.update({
                    # 'merged_chunks': current_chunk.metadata.get('merged_chunks', 1) + 1,
                    'chunk_length': len(current_chunk.page_content),
                    # 'is_merged': True
                })
                
                # 추가 법률 구조 정보가 있으면 업데이트
                if split.metadata.get('legal_article') and not current_chunk.metadata.get('legal_article'):
                    current_chunk.metadata['legal_article'] = split.metadata.get('legal_article')
                if split.metadata.get('legal_chapter') and not current_chunk.metadata.get('legal_chapter'):
                    current_chunk.metadata['legal_chapter'] = split.metadata.get('legal_chapter')
                if split.metadata.get('legal_section') and not current_chunk.metadata.get('legal_section'):
                    current_chunk.metadata['legal_section'] = split.metadata.get('legal_section')
                
                # # 항 정보는 누적
                # current_paragraphs = current_chunk.metadata.get('legal_paragraph', [])
                # new_paragraphs = split.metadata.get('legal_paragraph', [])
                # if isinstance(current_paragraphs, list) and isinstance(new_paragraphs, list):
                #     current_chunk.metadata['legal_paragraph'] = list(set(current_paragraphs + new_paragraphs))
                
            else:
                # 현재 청크가 충분히 크면 저장하고 새로운 청크 시작
                current_chunk.metadata['chunk_length'] = len(current_chunk.page_content)
                merged_splits.append(current_chunk)
                current_chunk = split
    
    # 마지막 청크 처리
    if current_chunk is not None:
        current_chunk.metadata['chunk_length'] = len(current_chunk.page_content)
        merged_splits.append(current_chunk)
    
    # 최종 청크 인덱스 재정렬
    for i, split in enumerate(merged_splits):
        # split.metadata['final_chunk_index'] = i
        split.metadata['page_chunk_id'] = f"page_{page_num}_chunk_{i}"
    
    return merged_splits

In [23]:
all_splits = []
error_files = []
for idx, file in enumerate(files):
    if file.endswith(".pdf") ==  False:
        continue
    FILE_PATH = "/Users/hongbikim/Dev/dacon_finance/docs/" + file
    file_id = uuid.uuid4().hex

    # PyMuPDF 로더 인스턴스 생성
    loader = PyMuPDFLoader(FILE_PATH)

    # 문서 로드
    docs = loader.load()

    for doc in docs:
        # 페이지별로 처리
        page_num = doc.metadata.get('page', 0)
        content = clean_text(doc.page_content)
        
        # 조문/장절 정보 추출
        legal_structure = extract_legal_structure(content)
        
        # 텍스트 분할
        splits = base_splitter.split_text(content)
        
        # 임시로 분할된 청크들 생성
        temp_splits = []
        for i, split_content in enumerate(splits):
            chunk_structure = extract_legal_structure(split_content)
            
            split_doc = Document(
                page_content=split_content,
                metadata={
                    # **doc.metadata,
                    # 'chunk_index': i,
                    'file_name': FILE_PATH.split("/")[-1],
                    'file_id': file_id,
                    'page_chunk_id': f"page_{page_num}_chunk_{i}",
                    # 'legal_article': chunk_structure.get('article'),
                    # 'legal_chapter': chunk_structure.get('chapter'),
                    # 'legal_section': chunk_structure.get('section'),
                    # 'legal_paragraph': chunk_structure.get('paragraph'),
                }
            )
            temp_splits.append(split_doc)
        
        # 짧은 청크들 합치기
        merged_splits = _merge_short_chunks(temp_splits, page_num)
        try:
            print(f"[{idx}/{len(files)}]")
            vector_store.add_documents(documents=merged_splits)
        except Exception as e:
            print(f"Error adding documents for file {i}th {file}: {e}")
            error_files.append(file)
            continue
        all_splits.extend(merged_splits)

[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[2/106]
[2/106]
[2/106]
[2/106]
[2/106]
[2/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]


In [12]:
all_splits

[Document(metadata={'file_name': '신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf', 'file_id': 'b06b545c29904d97ad4ffde6bc05c93d', 'page_chunk_id': 'page_0_chunk_0', 'chunk_length': 286}, page_content='법제처 1 국가법령정보센터\n신용정보의 이용 및 보호에 관한 법률 신용정보의 이용 및 보호에 관한 법률 ( 약칭: 신용정보법 )\n \n금융위원회 (금융데이터정책과) 02-2100-2624 제1장 총칙 제1조(목적) 이 법은 신용정보 관련 산업을 건전하게 육성하고 신용정보의 효율적 이용과 체계적 관리를 도모하며 신용\n정보의 오용ㆍ남용으로부터 사생활의 비밀 등을 적절히 보호함으로써 건전한 신용질서를 확립하고 국민경제의 발전\n에 이바지함을 목적으로 한다.  제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.'),
 Document(metadata={'file_name': '신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf', 'file_id': 'b06b545c29904d97ad4ffde6bc05c93d', 'page_chunk_id': 'page_0_chunk_1', 'chunk_length': 331}, page_content='1. “신용정보”란 금융거래 등 상거래에서 거래 상대방의 신용을 판단할 때 필요한 정보로서 다음 각 목의 정보를 말\n한다.\n가. 특정 신용정보주체를 식별할 수 있는 정보(나목부터 마목까지의 어느 하나에 해당하는 정보와 결합되는 경우\n만 신용정보에 해당한다)\n나. 신용정보주체의 거래내용을 판단할 수 있는 정보\n다. 신용정보주체의 신용도를 판단할 수 있는 정보\n라. 신용정보주체의 신용거래능력을 판단할 수 있는 정보\n마. 가목부터 라목까지의 정보 외에 신용정보주체의 신용을 판단할 때 필요한 정보\n1의2. 제1호가목의 “특정 

In [13]:
print(len(files))
print(len(all_splits))

106
10508


In [14]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "nlpai-lab/KURE-v1"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

/Users/hongbikim/anaconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Qdrant 클라이언트 생성 (메모리 기반)
# ":memory:" 옵션을 사용하면 휘발성(In-Memory) 데이터베이스로 실행됨.
# client = QdrantClient(":memory:")

client = QdrantClient(
    host="localhost",
    port=6333,
    # path = "/Users/hongbikim/Dev/dacon_finance"
)

# # 1. 먼저 컬렉션 생성 (임베딩 차원에 맞춰서)
# collection_name = "rag-finance"

# # 임베딩 차원 확인 (hf_embeddings의 차원)
# sample_embedding = hf_embeddings.embed_query("test")
# embedding_dim = len(sample_embedding)
# print(f"임베딩 차원: {embedding_dim}")

# # 컬렉션 생성
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=VectorParams(
#         size=embedding_dim,  # 임베딩 차원
#         distance=Distance.COSINE  # 거리 측정 방식
#     )
# )

# # Qdrant 벡터 저장소 객체 생성
# vector_store = QdrantVectorStore(
#     client = client,
#     collection_name = "rag-finance",
#     embedding = hf_embeddings
# )

In [21]:
def reset_collection(collection_name="rag-finance"):
    """컬렉션 초기화"""
    try:
        # 기존 컬렉션 삭제
        client.delete_collection(collection_name)
        print(f"✅ 컬렉션 '{collection_name}' 삭제 완료")
    except Exception as e:
        print(f"⚠️  컬렉션 삭제 중 오류: {e}")
    
    # 임베딩 차원 확인
    sample_embedding = hf_embeddings.embed_query("test")
    embedding_dim = len(sample_embedding)
    
    # 새 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=embedding_dim,
            distance=Distance.COSINE
        )
    )
    print(f"✅ 새 컬렉션 '{collection_name}' 생성 완료")
    
    # 새 벡터스토어 반환
    return QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=hf_embeddings
    )

# # 사용법
vector_store = reset_collection()

✅ 컬렉션 'rag-finance' 삭제 완료
✅ 새 컬렉션 'rag-finance' 생성 완료


In [126]:
vector_store.add_documents(documents=all_splits)

['0ab8dedf6933470db3c6fb746f416603',
 '81320a5b2df241c197d3b59524b4b264',
 'e1c96f2024bc4e12ada58fa4639b7670',
 '915fc725007c4bb6bc86018e32913cf5',
 'c72575ad9c6b40e79761cc68a8eea202',
 '23ef8323bb0d43a488ea4b16c20e3f73',
 '0b3ce69c416d4abda7d3bab6d7387fbf',
 'a363f2b8fc574c258bd848f8339fe7fe',
 'a05933c5e2944a179153960adc428a89',
 'b99584526f904a63910d845599ebf565',
 '058e3de425d848b0a70429e1d73e330e',
 '4cdaf4020087426fab860e22b3581017',
 'bd04aa4e48644a97a91afff46eeaae9d',
 '755516c2aa394da7b2262adc51d94cf6',
 '78a967bab3804014ba6a44b315f2ec17',
 '45537288ac0e412b87297873b225505d',
 'dafff347cdb74bcb8860581fdb7d2481',
 '44b7984d7182485682e53d27e4241c49',
 '823e3c2860714eb783aeecf71521b11a',
 'ab765d285b424329b1ec78f8063bfacd',
 '2e6a9454cd1c4149b2292ae37f9c3d6a',
 '7e2430a94e8b42fb81384bfbdf1e83a5',
 'f7a20aa776b94c06bb3ee4bc3bcadfa9',
 'd1a197c6f6b441738aee235e005ef6a7',
 '8c440f4bdda64cd29712cdb39241a471',
 '1ac862c1179546f1a6b2313764f3f5fb',
 '28c8dba5eb274655bdbf2330bb2af8fb',
 

In [26]:
collection_info = client.get_collection("rag-finance")
print(len(all_splits))
print(f"포인터 개수: {collection_info.points_count}")
print(f"컬렉션 상태: {collection_info.status}")



10508
포인터 개수: 10508
컬렉션 상태: green


In [27]:
test = pd.read_csv("/Users/hongbikim/Dev/dacon_finance/open/test.csv")

In [ ]:
q_i = 0
query = test['Question'][q_i].split("\n")[0]
# print(test['Question'][q_i])
# print(query)

results = vector_store.similarity_search(
    query = query,
    k = 10,
    # filter = models.Filter(must=[models.FieldCondition(
    #     key = "metadata.page", # 특정 필드(예: page) 기반 필터링
    #     match = models.MatchValue(value=2), # page 1인 문서만 검색
    # )])
)

ctxs = []
for result in results:
    # print(result.metadata['file_name'])
    ctxs.append(result.page_content)
ctxs

In [32]:
test['Question'][q_i]

'금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업\n2 투자자문업\n3 투자매매업\n4 투자중개업\n5 보험중개업'

In [ ]:
result.metadata['file_name']

In [37]:
ref = ""
for c_i, ctx in enumerate(ctxs):
    ref += str(c_i+1) + "번째 chunk\n" + ctx + "\n\n"
print(ref)

1번째 chunk: 법제처 20 국가법령정보센터
자본시장과 금융투자업에 관한 법률 시행령
1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목의 금융투자업의 경우: 해당 금융투자업에 속하는 금융투자상품
중 증권, 장내파생상품, 장외파생상품 각각을 기준으로 하여 인가받지 않은 다른 업무 단위
가. 투자매매업
나. 투자매매업(인수업은 제외한다)
다. 투자매매업(인수업만 해당한다)
2. 별표 1에 따른 금융투자업의 종류 중 투자중개업(같은 표에 따라 인가업무 단위가 2l-1-1, 2l-1-2, 2-14-1 및 2-
14-2인 투자중개업은 제외한다)의 경우: 투자중개업에 속하는 인가업무 단위 중 인가받지 않은 다른 업무 단위

2번째 chunk: 1. 금융투자업의 종류(투자매매업, 투자중개업, 집합투자업 및 신탁업을 말하되, 투자매매업 중 인수업을 포함한다)
2. 금융투자상품(집합투자업의 경우에는 제229조에 따른 집합투자기구의 종류를 말하며, 신탁업의 경우에는 제
103조제1항 각 호의 신탁재산을 말한다)의 범위(증권, 장내파생상품 및 장외파생상품을 말하되, 증권 중 국채증권
, 사채권, 그 밖에 대통령령으로 정하는 것을 포함하고 파생상품 중 주권을 기초자산으로 하는 파생상품ㆍ그 밖에
대통령령으로 정하는 것을 포함한다)
3. 투자자의 유형(전문투자자 및 일반투자자를 말한다. 이하 같다)

3번째 chunk: 법제처 9 국가법령정보센터
자본시장과 금융투자업에 관한 법률 시행령
무와 관련된 분석정보 등을 제공하는 경우
10. 다른 법령에 따라 건축물 및 주택의 임대관리 등 부동산의 관리대행, 부동산의 이용ㆍ개발 및 거래에 대한 상담,
그 밖에 부동산의 투자ㆍ운용에 관한 자문 등의 업무를 영위하는 경우

⑤ 법 제7조제6항에 따라 다음 각 호의 어느 하나에 해당하는 경우에는 해당 호의 금융투자업으로 보지 아니한다.

1. 법 제7조제6항제1호의 경우: 투자중개업
2. 법 제7조제6항제2호의 경우: 투자매매업
3. 법 제7조제6항제3호의 경우: 투자매매업 또는 투자중개업